In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

# Carica i dati puliti
print("Caricamento dei dati puliti...")
data = pd.read_csv('csv/cleaned_data.csv', parse_dates=['measure_date'])

# Riduzione del dataset
sample_size = min(150000, len(data))  # Usa un campione più piccolo se necessario
data_sampled = data.sample(n=sample_size, random_state=42)

print("Creazione della colonna target...")
threshold = data_sampled['value'].quantile(0.75)
data_sampled['target'] = (data_sampled['value'] > threshold).astype(int)

print("Distribuzione della variabile target:")
print(data_sampled['target'].value_counts(normalize=True))

# Feature engineering più efficiente
data_sampled['hour'] = data_sampled['measure_date'].dt.hour
data_sampled['day_of_week'] = data_sampled['measure_date'].dt.dayofweek
data_sampled['month'] = data_sampled['measure_date'].dt.month
data_sampled['is_weekend'] = data_sampled['day_of_week'].isin([5, 6]).astype(int)

# Lag features e rolling statistics (su un subset più piccolo di dati)
data_sampled = data_sampled.sort_values('measure_date')
data_sampled['value_lag_1'] = data_sampled.groupby('event_variable')['value'].shift(1)
data_sampled['value_rolling_mean'] = data_sampled.groupby('event_variable')['value'].rolling(window=24, min_periods=1).mean().reset_index(0, drop=True)

feature_columns = ['event_reference_value', 'hour', 'day_of_week', 'month', 'is_weekend', 
                   'value_lag_1', 'value_rolling_mean']

# Prepara i dati per il modello
X = data_sampled[feature_columns].dropna()
y = data_sampled['target'].loc[X.index]

# Divisione dei dati
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizzazione
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Bilanciamento delle classi (su un subset più piccolo se necessario)
if len(X_train) > 100000:
    X_train_subset, _, y_train_subset, _ = train_test_split(X_train_scaled, y_train, train_size=100000, stratify=y_train, random_state=42)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_subset, y_train_subset)
else:
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Ottimizzazione degli iperparametri con RandomizedSearchCV invece di GridSearchCV
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions, 
                                   n_iter=10, cv=3, n_jobs=-1, scoring='roc_auc', random_state=42)
random_search.fit(X_train_resampled, y_train_resampled)

print("Migliori parametri:", random_search.best_params_)
best_model = random_search.best_estimator_

# Valutazione del modello
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("\nReport di classificazione:")
print(classification_report(y_test, y_pred))

print("\nAUC-ROC Score:")
print(roc_auc_score(y_test, y_pred_proba))

# Matrice di confusione
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Matrice di Confusione')
plt.ylabel('Valore Reale')
plt.xlabel('Valore Predetto')
plt.savefig('plots/confusion_matrix.png')
plt.close()

# Importanza delle feature
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Importanza delle Feature')
plt.tight_layout()
plt.savefig('plots/feature_importance.png')
plt.close()

print("\nLe visualizzazioni sono state salvate nella cartella 'plots'.")
print("\nModellazione completata.")

Caricamento dei dati puliti...
Creazione della colonna target...
Distribuzione della variabile target:
target
0    0.7502
1    0.2498
Name: proportion, dtype: float64
Migliori parametri: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None}

Report di classificazione:
              precision    recall  f1-score   support

           0       0.78      0.79      0.78     22516
           1       0.33      0.31      0.32      7475

    accuracy                           0.67     29991
   macro avg       0.55      0.55      0.55     29991
weighted avg       0.66      0.67      0.67     29991


AUC-ROC Score:
0.6250019993214784

Le visualizzazioni sono state salvate nella cartella 'plots'.

Modellazione completata.


# Feature Engineering
- Lag features e rolling statistics:
Puoi espandere le lag features e le statistiche rolling per catturare trend e pattern a lungo termine.

In [3]:
import pandas as pd
import numpy as np

def create_advanced_features(df):
    # Ordina il dataframe per evento e data
    df = df.sort_values(['event_variable', 'measure_date'])
    
    # Lag features
    for lag in [1, 3, 7, 14, 30]:
        df[f'value_lag_{lag}'] = df.groupby('event_variable')['value'].shift(lag)
    
    # Differenze
    for lag in [1, 7]:
        df[f'value_diff_{lag}'] = df['value'] - df[f'value_lag_{lag}']
    
    # Rolling statistics
    for window in [24, 168, 720]:  # 1 day, 1 week, 1 month (assuming hourly data)
        df[f'value_rolling_mean_{window}'] = df.groupby('event_variable')['value'].rolling(window=window, min_periods=1).mean().reset_index(0, drop=True)
        df[f'value_rolling_std_{window}'] = df.groupby('event_variable')['value'].rolling(window=window, min_periods=1).std().reset_index(0, drop=True)
        df[f'value_rolling_max_{window}'] = df.groupby('event_variable')['value'].rolling(window=window, min_periods=1).max().reset_index(0, drop=True)
        df[f'value_rolling_min_{window}'] = df.groupby('event_variable')['value'].rolling(window=window, min_periods=1).min().reset_index(0, drop=True)
    
    # Expanding statistics
    df['value_expanding_mean'] = df.groupby('event_variable')['value'].expanding().mean().reset_index(0, drop=True)
    df['value_expanding_std'] = df.groupby('event_variable')['value'].expanding().std().reset_index(0, drop=True)
    
    # Rate of change
    df['value_rate_of_change'] = df.groupby('event_variable')['value'].pct_change()
    
    return df

# Uso della funzione
data = create_advanced_features(data)

- Interazioni tra feature:
Crea nuove feature che rappresentano interazioni tra le feature esistenti.

 il dataset contenga non solo valori NaN, ma anche valori infiniti o estremamente grandi. Dobbiamo gestire questi casi specifici. Ecco una versione aggiornata della funzione che dovrebbe gestire sia i valori NaN che quelli infiniti:

 Questa soluzione sostituisce i valori infiniti con NaN e poi imputa questi NaN con la media della colonna. 

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

def create_comprehensive_feature_interactions(df, degree=2, nan_threshold=0.05, manual_interactions=None):
    # Pulisci la colonna value_rate_of_change
    if 'value_rate_of_change' in df.columns:
        df['value_rate_of_change_clean'] = df['value_rate_of_change'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Riempi i valori NaN nelle colonne di lag
    for col in [c for c in df.columns if c.startswith('value_lag_')]:
        df[col] = df[col].fillna(df[col].mean())
    
    # Seleziona le colonne numeriche
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Rimuovi le colonne che non vuoi utilizzare per le interazioni
    columns_to_interact = [col for col in numeric_columns if col not in ['target', 'event_variable']]
    
    # Controllo dettagliato per valori infiniti e NaN
    clean_columns = []
    for col in columns_to_interact:
        inf_count = np.isinf(df[col]).sum()
        nan_count = np.isnan(df[col]).sum()
        total_count = len(df)
        print(f"Colonna {col}: {inf_count} infiniti, {nan_count} NaN")
        
        # Accetta colonne con meno del 5% di valori NaN e nessun infinito
        if inf_count == 0 and nan_count / total_count < nan_threshold:
            clean_columns.append(col)
    
    print(f"Colonne selezionate per le interazioni: {clean_columns}")
    
    if len(clean_columns) < 2:
        print("Non ci sono abbastanza colonne pulite per creare interazioni.")
        return df
    
    # Gestisci i valori NaN nelle colonne selezionate
    df_clean = df[clean_columns].copy()
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), columns=clean_columns, index=df.index)
    
    # Crea le interazioni
    try:
        poly = PolynomialFeatures(degree=degree, include_bias=False, interaction_only=True)
        interactions = poly.fit_transform(df_imputed)
        
        # Crea nomi per le nuove colonne
        interaction_names = poly.get_feature_names(clean_columns)
        
        # Aggiungi le nuove colonne al dataframe
        interaction_df = pd.DataFrame(interactions, columns=interaction_names, index=df.index)
        
        # Concatena le nuove colonne al dataframe originale
        df = pd.concat([df, interaction_df], axis=1)
        print(f"Interazioni create con successo. Nuove dimensioni del dataframe: {df.shape}")
    except Exception as e:
        print(f"Errore durante la creazione delle interazioni: {e}")
    
    # Crea interazioni manuali se specificate
    if manual_interactions:
        for interaction_name, (col1, col2) in manual_interactions.items():
            if col1 in df.columns and col2 in df.columns:
                df[interaction_name] = df[col1] * df[col2]
                print(f"Interazione manuale creata: {interaction_name}")
            else:
                print(f"Impossibile creare l'interazione {interaction_name}: colonne non trovate")
    
    return df

# Uso della funzione
manual_interactions = {
    'interaction_value_reference': ('value', 'event_reference_value'),
    'interaction_hour_day': ('hour', 'day_of_week')
}

data = create_comprehensive_feature_interactions(data, manual_interactions=manual_interactions)

# Visualizza le statistiche descrittive di alcune colonne
columns_to_describe = ['value', 'event_reference_value', 'value_rate_of_change_clean']
for col in columns_to_describe:
    if col in data.columns:
        print(f"\nStatistiche descrittive per {col}:")
        print(data[col].describe())
    else:
        print(f"\nColonna {col} non trovata nel dataset.")

Colonna value: 0 infiniti, 0 NaN
Colonna event_id: 0 infiniti, 0 NaN
Colonna event_reference_value: 0 infiniti, 0 NaN
Colonna hour: 0 infiniti, 0 NaN
Colonna day_of_week: 0 infiniti, 0 NaN
Colonna month: 0 infiniti, 0 NaN
Colonna is_weekend: 0 infiniti, 0 NaN
Colonna day_of_year: 0 infiniti, 0 NaN
Colonna week_of_year: 0 infiniti, 0 NaN
Colonna quarter: 0 infiniti, 0 NaN
Colonna hour_sin: 0 infiniti, 0 NaN
Colonna hour_cos: 0 infiniti, 0 NaN
Colonna day_of_week_sin: 0 infiniti, 0 NaN
Colonna day_of_week_cos: 0 infiniti, 0 NaN
Colonna month_sin: 0 infiniti, 0 NaN
Colonna month_cos: 0 infiniti, 0 NaN
Colonna value_lag_1: 0 infiniti, 0 NaN
Colonna value_lag_3: 0 infiniti, 0 NaN
Colonna value_lag_7: 0 infiniti, 0 NaN
Colonna value_lag_14: 0 infiniti, 0 NaN
Colonna value_lag_30: 0 infiniti, 0 NaN
Colonna value_diff_1: 0 infiniti, 45 NaN
Colonna value_diff_7: 0 infiniti, 315 NaN
Colonna value_rolling_mean_24: 0 infiniti, 0 NaN
Colonna value_rolling_std_24: 0 infiniti, 45 NaN
Colonna value_ro

La pulizia dei dati ha avuto successo:

La colonna value_rate_of_change_clean non ha più valori infiniti o NaN.
Le colonne di lag sono state riempite correttamente.


C'è stato un errore durante la creazione delle interazioni polinomiali:
CopyErrore durante la creazione delle interazioni: 'PolynomialFeatures' object has no attribute 'get_feature_names'
Questo errore suggerisce che stiamo usando una versione di scikit-learn che non ha il metodo get_feature_names. Dobbiamo aggiornare il codice per gestire questo.
Le interazioni manuali sono state create con successo.
Le statistiche descrittive mostrano che i dati hanno una grande variabilità, con alcuni valori molto alti (ad esempio, max per value è 800000).


Principali modifiche e miglioramenti:

- Abbiamo aggiunto una normalizzazione dei dati usando StandardScaler prima di creare le interazioni. Questo aiuta a gestire la grande variabilità nei dati.
- Abbiamo aggiornato il codice per gestire sia le versioni più recenti di scikit-learn (che usano get_feature_names_out) sia le versioni più vecchie.
- Abbiamo aggiunto una visualizzazione delle prime righe delle nuove colonne di interazione alla fine.
La funzione ora gestisce meglio potenziali errori durante la creazione delle interazioni.

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.impute import SimpleImputer

def create_updated_feature_interactions(df, degree=2, nan_threshold=0.05, manual_interactions=None):
    # Pulisci la colonna value_rate_of_change
    if 'value_rate_of_change' in df.columns:
        df['value_rate_of_change_clean'] = df['value_rate_of_change'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Riempi i valori NaN nelle colonne di lag
    for col in [c for c in df.columns if c.startswith('value_lag_')]:
        df[col] = df[col].fillna(df[col].mean())
    
    # Seleziona le colonne numeriche
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Rimuovi le colonne che non vuoi utilizzare per le interazioni
    columns_to_interact = [col for col in numeric_columns if col not in ['target', 'event_variable']]
    
    # Controllo dettagliato per valori infiniti e NaN
    clean_columns = []
    for col in columns_to_interact:
        inf_count = np.isinf(df[col]).sum()
        nan_count = np.isnan(df[col]).sum()
        total_count = len(df)
        print(f"Colonna {col}: {inf_count} infiniti, {nan_count} NaN")
        
        # Accetta colonne con meno del 5% di valori NaN e nessun infinito
        if inf_count == 0 and nan_count / total_count < nan_threshold:
            clean_columns.append(col)
    
    print(f"Colonne selezionate per le interazioni: {clean_columns}")
    
    if len(clean_columns) < 2:
        print("Non ci sono abbastanza colonne pulite per creare interazioni.")
        return df
    
    # Gestisci i valori NaN nelle colonne selezionate
    df_clean = df[clean_columns].copy()
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), columns=clean_columns, index=df.index)
    
    # Normalizza i dati
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=clean_columns, index=df.index)
    
    # Crea le interazioni
    try:
        poly = PolynomialFeatures(degree=degree, include_bias=False, interaction_only=True)
        interactions = poly.fit_transform(df_scaled)
        
        # Crea nomi per le nuove colonne
        if hasattr(poly, 'get_feature_names_out'):
            interaction_names = poly.get_feature_names_out(clean_columns)
        else:
            interaction_names = [f'interaction_{i}' for i in range(interactions.shape[1])]
        
        # Aggiungi le nuove colonne al dataframe
        interaction_df = pd.DataFrame(interactions, columns=interaction_names, index=df.index)
        
        # Concatena le nuove colonne al dataframe originale
        df = pd.concat([df, interaction_df], axis=1)
        print(f"Interazioni create con successo. Nuove dimensioni del dataframe: {df.shape}")
    except Exception as e:
        print(f"Errore durante la creazione delle interazioni: {e}")
    
    # Crea interazioni manuali se specificate
    if manual_interactions:
        for interaction_name, (col1, col2) in manual_interactions.items():
            if col1 in df.columns and col2 in df.columns:
                try:
                    # Assicuriamoci che stiamo lavorando con Series
                    series1 = df[col1] if isinstance(df[col1], pd.Series) else df[col1].iloc[:, 0]
                    series2 = df[col2] if isinstance(df[col2], pd.Series) else df[col2].iloc[:, 0]
                    
                    # Crea l'interazione
                    df[interaction_name] = series1 * series2
                    print(f"Interazione manuale creata: {interaction_name}")
                except Exception as e:
                    print(f"Errore nella creazione dell'interazione {interaction_name}: {e}")
                    print(f"Tipo di df[{col1}]: {type(df[col1])}")
                    print(f"Tipo di df[{col2}]: {type(df[col2])}")
                    if isinstance(df[col1], pd.DataFrame):
                        print(f"Colonne in df[{col1}]: {df[col1].columns}")
                    if isinstance(df[col2], pd.DataFrame):
                        print(f"Colonne in df[{col2}]: {df[col2].columns}")
            else:
                print(f"Impossibile creare l'interazione {interaction_name}: colonne non trovate")
                print(f"Colonne disponibili: {df.columns}")

    return df


data = pd.read_csv('csv/cleaned_data.csv', parse_dates=['measure_date'])
# Uso della funzione
manual_interactions = {
    'interaction_value_reference': ('value', 'event_reference_value'),
    'interaction_hour_day': ('hour', 'day_of_week')
}

data_sample = create_updated_feature_interactions(data, manual_interactions=manual_interactions)

# Visualizza le statistiche descrittive di alcune colonne
columns_to_describe = ['value', 'event_reference_value', 'value_rate_of_change_clean']
for col in columns_to_describe:
    if col in data.columns:
        print(f"\nStatistiche descrittive per {col}:")
        print(data[col].describe())
    else:
        print(f"\nColonna {col} non trovata nel dataset.")

# Visualizza le prime righe delle nuove colonne di interazione
interaction_columns = [col for col in data.columns if col.startswith('interaction_')]
print("\nPrime righe delle colonne di interazione:")
print(data[interaction_columns].head())

Colonna value: 0 infiniti, 0 NaN
Colonna event_id: 0 infiniti, 0 NaN
Colonna event_reference_value: 0 infiniti, 0 NaN
Colonne selezionate per le interazioni: ['value', 'event_id', 'event_reference_value']
Interazioni create con successo. Nuove dimensioni del dataframe: (6678924, 13)
Interazione manuale creata: interaction_value_reference
Impossibile creare l'interazione interaction_hour_day: colonne non trovate
Colonne disponibili: Index(['measure_name', 'value', 'measure_date', 'event_id', 'event_variable',
       'event_operator', 'event_reference_value', 'value', 'event_id',
       'event_reference_value', 'value event_id',
       'value event_reference_value', 'event_id event_reference_value',
       'interaction_value_reference'],
      dtype='object')

Statistiche descrittive per value:
count    6.678924e+06
mean     1.100769e+03
std      2.237396e+04
min     -2.620500e+04
25%      1.000000e+00
50%      3.000000e+00
75%      4.710000e+01
max      8.000000e+05
Name: value, dtype: 

- Lavora su un campione più piccolo dei dati (impostato a 100,000 righe di default).
- Seleziona solo le prime 10 colonne numeriche per le interazioni.
- Processa i dati in batch per ridurre l'uso della memoria.
- Riduce il numero di operazioni di pulizia e preprocessing.

Grazie per aver condiviso l'output dettagliato. Analizziamo i risultati:

1. Dati originali:
   - Il dataset originale contiene 6,678,924 righe e 7 colonne.
   - Le colonne originali sono: 'measure_name', 'value', 'measure_date', 'event_id', 'event_variable', 'event_operator', 'event_reference_value'.

2. Feature engineering:
   - Sono state create con successo nuove feature temporali (hour, day_of_week, month, is_weekend).
   - Sono state applicate trasformazioni logaritmiche a 'value' e 'event_reference_value'.
   - Non ci sono valori infiniti o NaN nelle colonne selezionate per le interazioni.

3. Interazioni:
   - Sono state create interazioni polinomiali tra le colonne selezionate.
   - Le interazioni manuali 'interaction_value_reference' e 'interaction_hour_day' sono state create con successo.

4. Dimensioni finali:
   - Il dataframe finale ha 58 colonne, indicando che sono state create 51 nuove feature.

5. Statistiche descrittive:
   - Le statistiche per 'value' e 'event_reference_value' mostrano una grande variabilità nei dati.
   - Le trasformazioni logaritmiche ('value_log' e 'event_reference_value_log') hanno ridotto questa variabilità.

6. Colonna mancante:
   - 'value_rate_of_change_clean' non è presente nel dataset finale. Questo potrebbe essere dovuto al fatto che 'value_rate_of_change' non era presente nel dataset originale.

Osservazioni e suggerimenti:

1. Il processo di feature engineering sembra essere stato eseguito con successo, creando numerose nuove feature senza introdurre valori problematici (infiniti o NaN).

2. La grande quantità di nuove feature (51) potrebbe portare a problemi di dimensionalità elevata. Considera di utilizzare tecniche di selezione delle feature o di riduzione della dimensionalità in fase di modellazione.

3. Le interazioni manuali sono state create correttamente, ma potresti voler verificare se 'interaction_hour_day' ha il comportamento desiderato, dato che i suoi valori sembrano essere tutti 132 nelle prime righe.

4. La mancanza di 'value_rate_of_change_clean' suggerisce che potresti voler aggiungere questa feature se è rilevante per la tua analisi.

5. Dato che il dataset è abbastanza grande (6,678,924 righe), assicurati di avere risorse computazionali sufficienti per gestirlo nelle fasi successive dell'analisi.

6. Potresti considerare di aggiungere alcune feature di lag o rolling statistics se sono rilevanti per il tuo problema.

Nel complesso, il processo di feature engineering sembra essere stato eseguito con successo, creando un ricco set di feature per l'analisi successiva. Il prossimo passo potrebbe essere la selezione delle feature più rilevanti e l'inizio della fase di modellazione.

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.impute import SimpleImputer

def safe_log1p(x):
    return np.log1p(np.maximum(x, 0))

def create_interaction(df, col1, col2):
    try:
        # Assicuriamoci che stiamo lavorando con Series
        series1 = df[col1] if isinstance(df[col1], pd.Series) else df[col1].iloc[:, 0]
        series2 = df[col2] if isinstance(df[col2], pd.Series) else df[col2].iloc[:, 0]
        
        # Converti in float e gestisci eventuali valori non numerici
        series1 = pd.to_numeric(series1, errors='coerce')
        series2 = pd.to_numeric(series2, errors='coerce')
        
        # Crea l'interazione
        result = series1 * series2
        
        # Assicuriamoci che il risultato sia una Series
        if isinstance(result, pd.DataFrame):
            result = result.iloc[:, 0]
        
        return result
    except Exception as e:
        print(f"Errore nella creazione dell'interazione tra {col1} e {col2}: {e}")
        return pd.Series(index=df.index)  # Restituisce una Series vuota in caso di errore

def create_robust_improved_feature_interactions(df, degree=2, nan_threshold=0.05, manual_interactions=None):
    # Aggiungi feature temporali
    df['hour'] = df['measure_date'].dt.hour
    df['day_of_week'] = df['measure_date'].dt.dayofweek
    df['month'] = df['measure_date'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

    # Pulisci la colonna value_rate_of_change
    if 'value_rate_of_change' in df.columns:
        df['value_rate_of_change_clean'] = df['value_rate_of_change'].replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # Applica trasformazione logaritmica alle colonne con grande variabilità
    for col in ['value', 'event_reference_value']:
        if col in df.columns:
            df[f'{col}_log'] = safe_log1p(df[col])
    
    # Rimuovi duplicati
    df = df.loc[:,~df.columns.duplicated()]
    
    # Seleziona le colonne numeriche
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Rimuovi le colonne che non vuoi utilizzare per le interazioni
    columns_to_interact = [col for col in numeric_columns if col not in ['target', 'event_variable']]
    
    # Controllo dettagliato per valori infiniti e NaN
    clean_columns = []
    for col in columns_to_interact:
        inf_count = np.isinf(df[col]).sum()
        nan_count = np.isnan(df[col]).sum()
        total_count = len(df)
        print(f"Colonna {col}: {inf_count} infiniti, {nan_count} NaN")
        
        # Accetta colonne con meno del 5% di valori NaN e nessun infinito
        if inf_count == 0 and nan_count / total_count < nan_threshold:
            clean_columns.append(col)
    
    # Filtra le colonne con troppi NaN
    clean_columns = [col for col in clean_columns if df[col].isna().sum() / len(df) < nan_threshold]
    
    print(f"Colonne selezionate per le interazioni: {clean_columns}")
    
    if len(clean_columns) < 2:
        print("Non ci sono abbastanza colonne pulite per creare interazioni.")
        return df
    
    # Gestisci i valori NaN nelle colonne selezionate
    df_clean = df[clean_columns].copy()
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(imputer.fit_transform(df_clean), columns=clean_columns, index=df.index)
    
    # Normalizza i dati
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed), columns=clean_columns, index=df.index)
    
    # Crea le interazioni
    try:
        poly = PolynomialFeatures(degree=degree, include_bias=False, interaction_only=True)
        interactions = poly.fit_transform(df_scaled)
        
        # Crea nomi per le nuove colonne
        if hasattr(poly, 'get_feature_names_out'):
            interaction_names = poly.get_feature_names_out(clean_columns)
        else:
            interaction_names = [f'interaction_{i}' for i in range(interactions.shape[1])]
        
        # Aggiungi le nuove colonne al dataframe
        interaction_df = pd.DataFrame(interactions, columns=interaction_names, index=df.index)
        
        # Concatena le nuove colonne al dataframe originale
        df = pd.concat([df, interaction_df], axis=1)
        print(f"Interazioni create con successo. Nuove dimensioni del dataframe: {df.shape}")
    except Exception as e:
        print(f"Errore durante la creazione delle interazioni: {e}")
    
    # Crea interazioni manuali se specificate
    if manual_interactions:
        for interaction_name, (col1, col2) in manual_interactions.items():
            if col1 in df.columns and col2 in df.columns:
                interaction = create_interaction(df, col1, col2)
                if not interaction.empty:
                    df[interaction_name] = interaction
                    print(f"Interazione manuale creata: {interaction_name}")
                else:
                    print(f"Impossibile creare l'interazione {interaction_name}: risultato vuoto")
            else:
                print(f"Impossibile creare l'interazione {interaction_name}: colonne non trovate")
                print(f"Colonne disponibili: {df.columns}")

    # Rimuovi duplicati finali
    df = df.loc[:,~df.columns.duplicated()]
    
    return df

# Carica i dati
print("Caricamento dei dati...")
data = pd.read_csv('csv/cleaned_data.csv', parse_dates=['measure_date'])

# Stampa informazioni sui dati originali
print("\nInformazioni sui dati originali:")
print(data.info())
print("\nColonne nel dataset originale:")
print(data.columns)
print("\nTipi di dati delle colonne originali:")
print(data.dtypes)

# Usa la funzione
print("\nCreazione delle feature interactions...")
manual_interactions = {
    'interaction_value_reference': ('value', 'event_reference_value'),
    'interaction_hour_day': ('hour', 'day_of_week')
}

data_sample = create_robust_improved_feature_interactions(data, manual_interactions=manual_interactions)

# Visualizza le statistiche descrittive di alcune colonne
columns_to_describe = ['value', 'event_reference_value', 'value_rate_of_change_clean', 'value_log', 'event_reference_value_log']
for col in columns_to_describe:
    if col in data_sample.columns:
        print(f"\nStatistiche descrittive per {col}:")
        print(data_sample[col].describe())
    else:
        print(f"\nColonna {col} non trovata nel dataset.")

# Visualizza le prime righe delle nuove colonne di interazione
interaction_columns = [col for col in data_sample.columns if 'interaction_' in col]
if interaction_columns:
    print("\nPrime righe delle colonne di interazione:")
    print(data_sample[interaction_columns].head())
else:
    print("\nNessuna colonna di interazione trovata.")

# Stampa informazioni aggiuntive per il debug
print("\nColonne nel dataframe finale:")
print(data_sample.columns)
print("\nTipi di dati delle colonne finali:")
print(data_sample.dtypes)

# Salva il risultato
print("\nSalvataggio del dataframe con le nuove feature...")
data_sample.to_csv('csv/data_with_interactions.csv', index=False)
print("Dataframe salvato con successo.")

Caricamento dei dati...

Informazioni sui dati originali:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6678924 entries, 0 to 6678923
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   measure_name           object        
 1   value                  float64       
 2   measure_date           datetime64[ns]
 3   event_id               int64         
 4   event_variable         object        
 5   event_operator         object        
 6   event_reference_value  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 356.7+ MB
None

Colonne nel dataset originale:
Index(['measure_name', 'value', 'measure_date', 'event_id', 'event_variable',
       'event_operator', 'event_reference_value'],
      dtype='object')

Tipi di dati delle colonne originali:
measure_name                     object
value                           float64
measure_date             datetime64[ns]
event_id     

# ottimizzazione del modello, la gestione dello sbilanciamento delle classi e la feature selection

Questo codice implementa:

## Ottimizzazione del Modello:

- Sperimenta con RandomForest, XGBoost e LightGBM.
- Usa RandomizedSearchCV per un tuning più approfondito degli iperparametri.
- Utilizza una cross-validation stratificata per una valutazione più robusta.


## Gestione dello Sbilanciamento delle Classi:

- Usa SMOTE per il oversampling della classe minoritaria.
- Puoi facilmente sostituire SMOTE con altre tecniche come RandomUnderSampler.


## Feature Selection:

Utilizza Recursive Feature Elimination (RFE) per selezionare le 20 feature più importanti.


## Valutazione del Modello:

- Calcola e stampa il report di classificazione e l'AUC-ROC score per ogni modello.
- Visualizza l'importanza delle feature per i modelli che lo supportano.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

def create_target(data):
    if 'target' not in data.columns:
        print("Creazione della colonna target...")
        threshold = data['value'].quantile(0.75)
        data['target'] = (data['value'] > threshold).astype(int)
    return data

def prepare_data(data, is_original=True):
    data = create_target(data)
    columns_to_drop = ['target', 'measure_date']
    if is_original:
        columns_to_drop.extend(['measure_name', 'event_variable', 'event_operator'])
    X = data.drop(columns_to_drop, axis=1, errors='ignore')
    y = data['target']
    return X, y

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    print("\nReport di classificazione:")
    print(classification_report(y_test, y_pred))
    print("\nAUC-ROC Score:")
    print(roc_auc_score(y_test, y_pred_proba))

def optimize_models(X, y, dataset_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    models = {
        'RandomForest': (RandomForestClassifier(random_state=42), {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [10, 20, None],
            'classifier__min_samples_split': [2, 5],
            'classifier__min_samples_leaf': [1, 2]
        }),
        'XGBoost': (XGBClassifier(random_state=42), {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [3, 5],
            'classifier__learning_rate': [0.01, 0.1],
            'classifier__subsample': [0.8, 1.0],
            'classifier__colsample_bytree': [0.8, 1.0]
        }),
        'LightGBM': (LGBMClassifier(random_state=42), {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [3, 5],
            'classifier__learning_rate': [0.01, 0.1],
            'classifier__subsample': [0.8, 1.0],
            'classifier__colsample_bytree': [0.8, 1.0]
        })
    }

    for model_name, (model, param_dist) in models.items():
        print(f"\nOttimizzazione del modello {model_name} su {dataset_name}...")
        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('feature_selection', RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=10)),
            ('sampler', SMOTE(random_state=42)),
            ('classifier', model)
        ])

        random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, 
                                           n_iter=10, cv=StratifiedKFold(n_splits=3), 
                                           scoring='roc_auc', n_jobs=-1, random_state=42)
        random_search.fit(X_train, y_train)

        print("Migliori parametri:", random_search.best_params_)
        best_model = random_search.best_estimator_

        evaluate_model(best_model, X_test, y_test)

        if hasattr(best_model.named_steps['classifier'], 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': X.columns[best_model.named_steps['feature_selection'].support_],
                'importance': best_model.named_steps['classifier'].feature_importances_
            }).sort_values('importance', ascending=False)

            plt.figure(figsize=(10, 6))
            sns.barplot(x='importance', y='feature', data=feature_importance)
            plt.title(f'Feature Importance - {model_name} ({dataset_name})')
            plt.tight_layout()
            plt.savefig(f'plots/feature_importance_{model_name}_{dataset_name}.png')
            plt.close()

# Carica i dati originali
print("Caricamento dei dati originali...")
data_original = pd.read_csv('csv/cleaned_data.csv', parse_dates=['measure_date'])

# Carica i dati con feature engineering
print("Caricamento dei dati con feature engineering...")
data_engineered = pd.read_csv('csv/data_with_interactions.csv', parse_dates=['measure_date'])

# Campionamento di un sottoinsieme più piccolo per entrambi i dataset
sample_size = min(100000, len(data_original), len(data_engineered))
data_original_sampled = data_original.sample(n=sample_size, random_state=42)
data_engineered_sampled = data_engineered.sample(n=sample_size, random_state=42)

print(f"Dimensione del campione: {sample_size}")

# Prepara i dati e ottimizza i modelli per i dati originali
X_original, y_original = prepare_data(data_original_sampled, is_original=True)
optimize_models(X_original, y_original, "Dati Originali")

# Prepara i dati e ottimizza i modelli per i dati con feature engineering
X_engineered, y_engineered = prepare_data(data_engineered_sampled, is_original=False)
optimize_models(X_engineered, y_engineered, "Dati con Feature Engineering")

print("\nOttimizzazione dei modelli completata per entrambi i dataset.")

Caricamento dei dati originali...
Caricamento dei dati con feature engineering...
Dimensione del campione: 100000
Creazione della colonna target...

Ottimizzazione del modello RandomForest su Dati Originali...


/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packa

Migliori parametri: {'classifier__n_estimators': 100, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 20}

Report di classificazione:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15003
           1       1.00      1.00      1.00      4997

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000


AUC-ROC Score:
1.0

Ottimizzazione del modello XGBoost su Dati Originali...


/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(
/Users/ele/Library/Python/3.12/lib/python/site-packa

Migliori parametri: {'classifier__subsample': 0.8, 'classifier__n_estimators': 100, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.1, 'classifier__colsample_bytree': 1.0}

Report di classificazione:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15003
           1       1.00      1.00      1.00      4997

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000


AUC-ROC Score:
0.9999980658927917

Ottimizzazione del modello LightGBM su Dati Originali...


...

/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=10 > n_features=3. There will be no feature selection and all features will be kept.
  warnings.warn(


[LightGBM] [Info] Number of positive: 60010, number of negative: 60010
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 643
[LightGBM] [Info] Number of data points in the train set: 120020, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ele/Library/Python/3.12/lib/python/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ele/Library/Python/3.12/lib/python/site-packages/imblearn/pipeline.py", line 329, in fit
    Xt, yt = self._fit(X, y, routed_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ele/Library/Python/3.12/lib/python/site-packages/imbl